# Meilisearch 

## Prepare Data

In [1]:
!pip install pandas psycopg "psycopg[binary]"
import psycopg
import pandas as pd

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached psycopg-3.2.3-py3-none-any.whl.metadata (4.3 kB)
  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached psycopg_binary-3.2.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (2.8 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Using cached psycopg-3.2.3-py3-none-any.whl (197 kB)
Using cached psycopg_binary-3.2.3-cp312-cp312-macosx_14_0_arm64.whl (3.5 MB)
Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is availa

In [2]:
from getpass import getpass
DATABASE_URL = getpass()

 ········


In [3]:
rows = []
async with await psycopg.AsyncConnection.connect(DATABASE_URL) as conn:
    async with conn.cursor() as cur:
        await cur.execute("""
            SELECT 
                store_id, 
                store_name, 
                brand, 
                address,
                ST_X(coordinates::geometry) AS longitude,
                ST_Y(coordinates::geometry) AS latitude
            FROM
                stores
        """)

        async for row in cur:
            rows.append(row)
            
        await conn.commit()

pd.DataFrame(rows, columns=["store_id", "store_name", "brand", "address", "longitude", "latitude"])

,store_id,store_name,brand,address,longitude,latitude
0,013115,全家新德店,FamilyMart,台北市中正區八德路一段４３巷２號,121.531597,25.044475
1,018527,全家台大醫院店,FamilyMart,台北市中正區中山南路７號Ｂ１樓,121.518534,25.040793
2,012555,全家新惠安店,FamilyMart,台北市中正區中華路二段３０７巷４６號,121.506467,25.029530
3,008805,全家南機場店,FamilyMart,台北市中正區中華路二段３１１巷１號１樓,121.505774,25.028401
4,018519,全家高鐵一店,FamilyMart,台北市中正區北平西路３號Ｂ１東側,121.517044,25.047203
...,...,...,...,...,...,...
11396,277181,金門大學,7-11,金門縣金寧鄉大學路1號1樓,118.322445,24.448280
11397,970820,金寧,7-11,金門縣金寧鄉環島北路一段711號,118.337676,24.449616
11398,215091,樂多,7-11,金門縣金寧鄉伯玉路二段229號,118.338985,24.440419
11399,173977,小金,7-11,金門縣烈嶼鄉林湖村東林街156號,118.247362,24.428631


## Meilisearch Indexing

```sh
# Fetch the latest version of Meilisearch image from DockerHub
docker pull getmeili/meilisearch:v1.11

# Launch Meilisearch in development mode with a master key
docker run -it --rm \
    -p 7700:7700 \
    -e MEILI_ENV='development' \
    -v $(pwd)/meili_data:/meili_data \
    getmeili/meilisearch:v1.11
```

In [4]:
!pip install meilisearch

  Using cached meilisearch-0.31.6-py3-none-any.whl.metadata (942 bytes)
  Using cached camel_converter-4.0.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached meilisearch-0.31.6-py3-none-any.whl (23 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)
Using cached camel_converter-4.0.1-py3-none-any.whl (6.2 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from getpass import getpass
MEILISEARCH_URL = getpass()
MEILI_MASTER_KEY = getpass()

 ········
 ········


In [7]:
import meilisearch

client = meilisearch.Client(MEILISEARCH_URL, MEILI_MASTER_KEY)

In [11]:
task = client.create_index('stores', { 'primaryKey': 'store_id' })
client.wait_for_task(task.task_uid)

docs = pd.DataFrame(rows, columns=["store_id", "store_name", "brand", "address", "longitude", "latitude"])[['store_id', 'store_name', 'address']].to_dict('records')
task = client.index('stores').add_documents(docs)
client.wait_for_task(task.task_uid)

Task(uid=1, index_uid='stores', status='succeeded', type='documentAdditionOrUpdate', details={'receivedDocuments': 11401, 'indexedDocuments': 11401}, error=None, canceled_by=None, duration='PT2.449832454S', enqueued_at=datetime.datetime(2024, 11, 8, 16, 12, 52, 261920), started_at=datetime.datetime(2024, 11, 8, 16, 12, 52, 264864), finished_at=datetime.datetime(2024, 11, 8, 16, 12, 54, 714696))

In [9]:
res = client.index('stores').search('板橋文化', opt_params={ 'showRankingScore': True, 'limit': 50, 'rankingScoreThreshold': 0.9 })

# res = pd.DataFrame(res['hits'], columns=["store_id", "store_name", "address", "_rankingScore"])

search_res = []
async with await psycopg.AsyncConnection.connect(DATABASE_URL) as conn:
    async with conn.cursor() as cur:
        await cur.execute("""
            SELECT 
                store_id, 
                store_name, 
                brand, 
                address,
                ST_X(coordinates::geometry) AS longitude,
                ST_Y(coordinates::geometry) AS latitude
            FROM
                stores
            WHERE
                store_id = ANY(%s)
            """,
            [[hit['store_id'] for hit in res['hits']]]
        )
        rows = await cur.fetchall()

        for row, hit in zip(rows, res['hits']):
            (store_id, store_name, brand, address, longitude, latitude) = row
            search_res.append({
                'store_id': store_id,
                'store_name': store_name,
                'brand': brand,
                'address': address,
                'longitude': longitude,
                'latitude': latitude,
                'score': hit['_rankingScore']
            })
            
        await conn.commit()


pd.DataFrame(search_res)

,store_id,store_name,brand,address,longitude,latitude,score
0,000469,全家板橋文化店,FamilyMart,新北市板橋區文化路一段２８０號,121.467183,25.021170,0.998592
1,002770,全家板橋新民店,FamilyMart,新北市板橋區文化路一段１８８巷４２號１樓,121.466741,25.018977,0.986271
2,006540,全家板橋龍江店,FamilyMart,新北市板橋區文化路二段１８２巷３弄９１，９３號１樓,121.472450,25.029126,0.986271
3,006738,全家板橋勝文店,FamilyMart,新北市板橋區仁化街４６號及４６之１號１樓,121.475206,25.031672,0.986271
4,012688,全家板橋文興店,FamilyMart,新北市板橋區民生路二段２３０號１樓,121.469063,25.022157,0.986271
5,013484,全家板橋興盛店,FamilyMart,新北市板橋區文聖街１６１號１樓,121.478933,25.027460,0.986271
6,015190,全家板橋立都店,FamilyMart,新北市板橋區文化路二段２５９號１樓,121.471268,25.029075,0.986271
7,015455,全家板橋致科店,FamilyMart,新北市板橋區文化路一段３１３號１樓,121.465214,25.020853,0.986271
8,018349,全家板橋傑仕堡店,FamilyMart,新北市板橋區文化路一段１８８巷５７號１樓（配送用）,121.468186,25.018240,0.986271
9,019020,全家板橋新埔店,FamilyMart,新北市板橋區文化路一段４２１巷４號１樓,121.467302,25.022844,0.986271
